In [120]:
import numpy as np
import pandas as pd
import seaborn as sns

In [121]:
path = r"E:\Metro1\\"

access_df=pd.read_csv(path+"access_data.csv")
egress_df=pd.read_csv(path+"egress_data.csv")



In [122]:
access_df.columns

Index(['ID', 'ACCESS_SLOPE', 'O_LEVEL', 'OM_LEVEL', 'A_LEVEL_DIFF',
       'A_SLOPE_TYPE', 'EGRESS_SLOPE', 'DM_LEVEL', 'D_LEVEL', 'E_LEVEL_DIFF',
       'E_SLOPE_TYPE', 'M_ORIGIN', 'M_DESTINATION', 'D_LANDMARK', 'D_LAT',
       'D_LONG', 'E_DISTANCE_MAP', 'E_TIME_MAP', 'D_TYPE', 'O_LANDMARK',
       'O_LAT', 'O_LONG', 'A_DISTANCE_MAP', 'A_TIME_MAP', 'O_TYPE',
       'START_TIME', 'TRIP_SEQUENCE', 'TOTAL_TIME', 'ACCESS_MODE',
       'ACCESS_TIME', 'ACCESS_DISTANCE', 'WT_FS', 'EGRESS_MODE', 'EGRESS_TIME',
       'EGRESS_DISTANCE', 'WT_FIS', 'BICYCLE_USE', 'TRANSFER_TIME',
       'WAITING_TIME_SECOND_STOP', 'PASS', 'TOTAL_COST', 'CROWDING', 'GENDER',
       'S_1', 'S_2', 'S_3', 'AGE', 'EDUCATION', 'OCCUPATION', 'INCOME', 'CARS',
       'TWO_WHEELER', 'N_BICYCLES', 'OTHERS', 'PARKING', 'PARK_RENT', 'PARK',
       'DESTINATION', 'ORIGIN', 'TRIP', 'AGE_C', 'ACCESS_DIS', 'EGRESS_DIS',
       'RENT_BICYCLE', 'MALE', 'ACCESS_M', 'EGRESS_M', 'ACCESS_DIS_B',
       'EGRESS_DIS_B', 'VEHICLE', 'M_O

In [123]:
access_df.shape[0]

# drop duplicate entries with same ID
access_df.drop_duplicates(subset=['ID'], inplace=True)

In [124]:
trip_types = {
    'Home based work': [('Home', 'Work'), ('Work', 'Home')],
    'Home based education': [('Home', 'School'), ('School', 'Home')],
    'Home based other': [('Home', 'Other'), ('Other', 'Home'),('Home',  'Restaurant'), ( 'Restaurant', 'Home'),
                         ('Home', 'Shopping'), ('Shopping', 'Home'),('Home', 'Social'), ('Social', 'Home'),
                         ('Home', 'Friend'), ('Friend', 'Home'),],
}


In [125]:
df=access_df.copy()
all_trips = set((row['O_TYPE'], row['D_TYPE']) for _, row in df.iterrows())
categorized_trips = set(trip for trips in trip_types.values() for trip in trips)
non_home_based_trips = all_trips - categorized_trips
trip_types['Non home based'] = list(non_home_based_trips)

# Calculate the number of trips and percentages
results = []
trip_totals = {}
trip_percentages = {}

for trip_type, pairs in trip_types.items():
    if trip_type == 'Non home based':
        total_trips = sum(len(df[(df['O_TYPE'] == origin) & (df['D_TYPE'] == destination)]) for origin, destination in non_home_based_trips)
        trip_totals[trip_type] = total_trips
        trip_percentage = total_trips / len(df) * 100 if len(df) > 0 else 0
        trip_percentages[trip_type] = round(trip_percentage,2)
        results.append([trip_type, 'Non home', 'Non home', total_trips])
    else:
        total_trips = sum(len(df[(df['O_TYPE'] == origin) & (df['D_TYPE'] == destination)]) for origin, destination in pairs)
        trip_totals[trip_type] = total_trips
        trip_percentage = total_trips / len(df) * 100 if len(df) > 0 else 0
        trip_percentages[trip_type] = trip_percentage
        for origin, destination in pairs:
            trip_count = len(df[(df['O_TYPE'] == origin) & (df['D_TYPE'] == destination)])
            results.append([trip_type, origin, destination, trip_count])

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['Type of trip', 'Origin', 'Destination', 'No of trips'])

# Calculate %OD based
total_trips = len(df)
results_df['%OD based'] = round(results_df['No of trips'] / total_trips * 100,2)

# Adding Sl No
results_df.insert(0, 'Sl No', range(1, len(results_df) + 1))

# Grouping by 'Type of trip'
grouped_df = results_df.groupby('Type of trip')

# Function to create a table with solid lines and centered text
def print_solid_table(df, trip_totals, trip_percentages):
    header = ["Sl No", "Type of trip", "Origin", "Destination", "No of trips", "%(type of trips)", "%OD based"]
    max_len = {col: max(df[col].astype(str).map(len).max(), len(col)) if col in df else len(col) for col in header}
    
    def draw_line():
        line = "+"
        for col in header:
            line += "-" * (max_len[col] + 2) + "+"
        print(line)

    def draw_row(row, show_type, show_sl_no, show_percentage):
        line = "|"
        for col in header:
            if col == "Type of trip" and not show_type:
                line += " " * (max_len[col] + 2) + "|"
            elif col == "Sl No" and not show_sl_no:
                line += " " * (max_len[col] + 2) + "|"
            elif col == "%(type of trips)" and not show_percentage:
                line += " " * (max_len[col] + 2) + "|"
            else:
                line += " " + str(row[col]).center(max_len[col]) + " |"
        print(line)

    def draw_merged_row(index, label, count, percentage):
        sl_no = str(index)
        line = "| " + sl_no.center(max_len["Sl No"]) + " | " + label.center(max_len["Type of trip"]) + " |"
        line += " " * (max_len["Origin"] + 2) + "|"
        line += " " * (max_len["Destination"] + 2) + "|"
        line += str(count).center(max_len["No of trips"] + 2) + "|"
        line += str(round(percentage, 2)).center(max_len["%(type of trips)"] + 2) + "|"
        line += " " * (max_len["%OD based"] + 2) + "|"
        print(line)

    draw_line()
    draw_row({col: col for col in header}, True, True, True)
    draw_line()
    
    sl_no = 1
    for trip_type, group in grouped_df:
        draw_merged_row(sl_no, trip_type, trip_totals[trip_type], trip_percentages[trip_type])
        type_trip_count = len(group)
        for i, (_, row) in enumerate(group.iterrows()):
            show_type = i < 0
            show_sl_no = i == -1
            draw_row(row, show_type, show_sl_no, False)
            type_trip_count -= 1
        draw_line()
        sl_no += 1

# Print the final table
print_solid_table(results_df, trip_totals, trip_percentages)

# Print the total number of samples
# print(f"\nTotal number of samples: {total_trips}")

+-------+----------------------+------------+-------------+-------------+------------------+-----------+
| Sl No |     Type of trip     |   Origin   | Destination | No of trips | %(type of trips) | %OD based |
+-------+----------------------+------------+-------------+-------------+------------------+-----------+
|   1   | Home based education |            |             |     141     |      16.97       |           |
|       |                      |    Home    |    School   |      73     |                  |    8.78   |
|       |                      |   School   |     Home    |      68     |                  |    8.18   |
+-------+----------------------+------------+-------------+-------------+------------------+-----------+
|   2   |   Home based other   |            |             |     272     |      32.73       |           |
|       |                      |    Home    |    Other    |      46     |                  |    5.54   |
|       |                      |   Other    |     Home 

In [126]:
# remove duplicate rows with same id column from dataframe
access_df.drop_duplicates(subset ="ID",inplace = True)
egress_df.drop_duplicates(subset ="ID",inplace = True)

print(access_df.shape,egress_df.shape)

# print(access_df.shape,egress_df.shape)
old_counts=access_df['RENT_BICYCLE'].value_counts()
print(old_counts)

replacement_mapping = {4: 2, 3: 1}
access_df['RENT_BICYCLE'] = access_df['RENT_BICYCLE'].replace(replacement_mapping)
new_counts = access_df['RENT_BICYCLE'].value_counts()
print(new_counts)



(831, 78) (711, 86)
RENT_BICYCLE
4    388
3    233
1    161
2     49
Name: count, dtype: int64
RENT_BICYCLE
2    437
1    394
Name: count, dtype: int64


In [127]:
dict={'Walk ':'Walk', 'Auto Rickshaw ':'Ride hire', 'Dropped-off by friends/others ':'Ride hire',
       'Two-Wheeler ':'Personal vehicle', 'Ola/Uber car ':'Personal vehicle', 'Bicycle ':'Personal vehicle', 'Car ':'Personal vehicle',
       'Rapido/Uber moto/Ola App two-wheeler ':'Ride hire', 'Shuttle':'Ride hire',
       'Rapido/uber moto/Ola App two-wheeler ':'Ride hire', 'Uber auto ':'Ride hire',
       'Company shuttle':'Ride hire', 'Office metro feeder':'Ride hire',
       'Dropped-off by friends/family members/others ':'Ride hire',
       'Drove and parked a two-Wheeler ':'Personal vehicle', 'Drove and parked a car ':'Personal vehicle',
       'Friends two wheeler':'Personal vehicle'}

access_df['ACCESS_MODE']=access_df['ACCESS_MODE'].map(dict)

In [128]:
access_df['AGE'].unique()

age_mapper={'> 60 yrs ':3, '18-25 yrs ':1, '26-35 yrs ':1, '18-25yrs ':1, '26-35yrs ':1,
       '46-60 yrs ':3, '36-45 yrs ':2, '36-45yrs ':2,'46-60yrs ':3,'> 60 yrs ':3, '18-25 yrs ':1, '26-35 yrs ':1, '26-35yrs ':1, '18-25yrs ':1,
       '46-60 yrs ':3, '36-45 yrs ':2, '36-45yrs ':2, '46-60yrs ':3}
access_df['AGE_C']=access_df['AGE'].map(age_mapper)
egress_df['AGE_C']=egress_df['17'].map(age_mapper)

print(access_df['AGE_C'].value_counts(),egress_df['AGE_C'].value_counts())

access_df['AGE1']=0
access_df['AGE2']=0
access_df['AGE3']=0

# if AGE_C is 1 then AGE1 is 1
access_df.loc[access_df['AGE_C']==1,'AGE1']=1
# if AGE_C is 2 then AGE2 is 1
access_df.loc[access_df['AGE_C']==2,'AGE2']=1
# if AGE_C is 3 then AGE3 is 1
access_df.loc[access_df['AGE_C']==3,'AGE3']=1

AGE_C
1    660
2    108
3     63
Name: count, dtype: int64 Series([], Name: count, dtype: int64)


In [129]:
access_df.columns

# rename few columns
access_df.rename(columns = {'A_DISTANCE_MAP':'DISTANCE', 'A_TIME_MAP':'TIME','ACCESS_SLOPE':'SLOPE',
                            'E_SLOPE_TYPE':'SLOPE_TYPE'}, inplace = True)

In [130]:
print(access_df[access_df['SLOPE']<-1].shape[0])
print(access_df[access_df['SLOPE']>1].shape[0])
print(access_df.shape[0]-276-166)

178
284
389


In [131]:
access_df['UP']=0
access_df['DOWN']=0
access_df['FLAT']=0

# if SLOPE is positive then make POS as 1 else 0
access_df.loc[access_df['SLOPE']>1,'UP']=1
access_df.loc[access_df['SLOPE']<-1,'DOWN']=1
access_df.loc[(access_df['SLOPE']>=-1) & (access_df['SLOPE']<=1),'FLAT']=1

access_df['VEH']=0

access_df['POS']=0

# if slope is greater than 0 then POS is 1 else 0
access_df.loc[access_df['SLOPE']>0,'POS']=1

# if CARS or TWO_WHEELER is greater than 1 then mark VEH as 1 else 0
access_df.loc[(access_df['CARS']>1) | (access_df['TWO_WHEELER']>1),'VEH']=1


access_df['SHORT']=0
access_df['MEDIUM']=0
access_df['LONG']=0

# if DISTANCE_MAP is less than 1 then make LOW as 1
access_df.loc[access_df['DISTANCE']<1,'SHORT']=1
# if DISTANCE_MAP is greater than 1 and less than 3 then make MEDIUM 1
access_df.loc[(access_df['DISTANCE']>=1) & (access_df['DISTANCE']<3),'MEDIUM']=1
# if DISTANCE_MAP is greater than 3 then make HIGH as 1
access_df.loc[access_df['DISTANCE']>=3,'LONG']=1

access_df['O_POP_DEN']=access_df['O_POP_DEN']/10000
access_df['O_EMP_DEN']=access_df['O_EMP_DEN']/10000
access_df['M_POP_DEN']=access_df['M_POP_DEN']/10000
access_df['M_EMP_DEN']=access_df['M_EMP_DEN']/10000




In [132]:
# convert 2 in RENT_BICYCLE to 4 and 3 to 1 in access_df dataframe
access_df['RENT_BICYCLE'] = access_df['RENT_BICYCLE'].replace({4: 2, 3: 1})

# create a new column WORK where if O_TYPE or D_TYPE is work then it is 1 else 0
access_df['WORK']=0
access_df.loc[(access_df['O_TYPE']=='Work') | (access_df['D_TYPE']=='Work'),'WORK']=1

access_df['MODE']=0


access_df['DEN']=0
access_df['DEN']=access_df['A_INTERSECTIONS']/access_df['DISTANCE']

access_df=access_df[access_df['DEN']<30]

access_df['UNO']=1

access_df['HIGH']=0
access_df['MED']=0

# if DEN is greater than 66 percentile of data of DEN then HIGH is 1 for that entry
# if DEN is between 33 and 66 percentiles of data of DEN then MED is 1 for that entry 
access_df.loc[access_df['DEN']>access_df['DEN'].quantile(0.66),'HIGH']=1
access_df.loc[(access_df['DEN']<=access_df['DEN'].quantile(0.66)) & (access_df['DEN']>access_df['DEN'].quantile(0.33)),'MED']=1

access_df['DIST1']=0
access_df['DIST2']=0
access_df['DIST3']=0
access_df['DIST4']=0
access_df['DIST5']=0

access_df.loc[access_df['DISTANCE']<1,'DIST1']=1
access_df.loc[(access_df['DISTANCE']>=1) & (access_df['DISTANCE']<2),'DIST2']=1
access_df.loc[(access_df['DISTANCE']>=2) & (access_df['DISTANCE']<3),'DIST3']=1
access_df.loc[(access_df['DISTANCE']>=3) & (access_df['DISTANCE']<4),'DIST4']=1
access_df.loc[(access_df['DISTANCE']>=4),'DIST5']=1

access_df['o_pop_high']=0
access_df['o_pop_med']=0
access_df['o_pop_low']=0

access_df.loc[access_df['O_POP_DEN']>access_df['O_POP_DEN'].quantile(0.66),'o_pop_high']=1
access_df.loc[(access_df['O_POP_DEN']<=access_df['O_POP_DEN'].quantile(0.66)) & (access_df['O_POP_DEN']>access_df['O_POP_DEN'].quantile(0.33)),'o_pop_med']=1
access_df.loc[access_df['O_POP_DEN']<=access_df['O_POP_DEN'].quantile(0.33),'o_pop_low']=1

access_df['o_emp_high']=0
access_df['o_emp_med']=0
access_df['o_emp_low']=0

access_df.loc[access_df['O_EMP_DEN']>access_df['O_EMP_DEN'].quantile(0.66),'o_emp_high']=1
access_df.loc[(access_df['O_EMP_DEN']<=access_df['O_EMP_DEN'].quantile(0.66)) & (access_df['O_EMP_DEN']>access_df['O_EMP_DEN'].quantile(0.33)),'o_emp_med']=1
access_df.loc[access_df['O_EMP_DEN']<=access_df['O_EMP_DEN'].quantile(0.33),'o_emp_low']=1

access_df['m_pop_high']=0
access_df['m_pop_med']=0
access_df['m_pop_low']=0

access_df.loc[access_df['M_POP_DEN']>access_df['M_POP_DEN'].quantile(0.66),'m_pop_high']=1
access_df.loc[(access_df['M_POP_DEN']<=access_df['M_POP_DEN'].quantile(0.66)) & (access_df['M_POP_DEN']>access_df['M_POP_DEN'].quantile(0.33)),'m_pop_med']=1
access_df.loc[access_df['M_POP_DEN']<=access_df['M_POP_DEN'].quantile(0.33),'m_pop_low']=1

access_df['m_emp_high']=0
access_df['m_emp_med']=0
access_df['m_emp_low']=0

access_df.loc[access_df['M_EMP_DEN']>access_df['M_EMP_DEN'].quantile(0.66),'m_emp_high']=1
access_df.loc[(access_df['M_EMP_DEN']<=access_df['M_EMP_DEN'].quantile(0.66)) & (access_df['M_EMP_DEN']>access_df['M_EMP_DEN'].quantile(0.33)),'m_emp_med']=1
access_df.loc[access_df['M_EMP_DEN']<=access_df['M_EMP_DEN'].quantile(0.33),'m_emp_low']=1




# output columns ID, RENT_BICYCLE,MALE, ACCESS_SLOPE, A_SLOPE_TYPE, A_DISTANCE_MAP, A_TIME_MAP,O_TYPE,D_TYPE,ACCESS_MODE,AGE_C,N_BICYCLES,CARS,
# TWO_WHEELER, OTHERS, A_INTERSECTINS from access_df dataframe to csv file
access_df[['ID', 'RENT_BICYCLE', 'MALE', 'SLOPE', 'SLOPE_TYPE', 'DISTANCE', 'TIME','O_TYPE','D_TYPE','ACCESS_MODE',
           'AGE1','AGE2','AGE3','VEH','WORK','UNO','SHORT','LONG','MEDIUM','UP','DOWN','POS','FLAT','DIST2','A_INTERSECTIONS',
           'o_pop_high','o_pop_med','o_pop_low','o_emp_high','o_emp_med','o_emp_low','m_pop_high','m_pop_med','m_pop_low',
             'm_emp_high','m_emp_med','m_emp_low','HIGH','MED','DIST5','DIST2','DEN',
        'DIST1','DIST3','DIST4','PARK_RENT','O_POP_DEN', 'O_EMP_DEN', 'M_POP_DEN', 'M_EMP_DEN']].to_csv(path+"access_data_analysis.csv",index=False)

In [133]:
print(access_df['DIST1'].value_counts(),access_df['DIST2'].value_counts(),access_df['DIST3'].value_counts(),access_df['DIST4'].value_counts(),access_df['DIST5'].value_counts())

DIST1
0    529
1    249
Name: count, dtype: int64 DIST2
0    574
1    204
Name: count, dtype: int64 DIST3
0    636
1    142
Name: count, dtype: int64 DIST4
0    710
1     68
Name: count, dtype: int64 DIST5
0    663
1    115
Name: count, dtype: int64


In [134]:
print(access_df['PARK_RENT'].value_counts(),access_df['MALE'].value_counts(),access_df['AGE_C'].value_counts(),access_df['WORK'].value_counts())

PARK_RENT
10    355
15    221
5     202
Name: count, dtype: int64 MALE
1    435
0    343
Name: count, dtype: int64 AGE_C
1    620
2     99
3     59
Name: count, dtype: int64 WORK
0    437
1    341
Name: count, dtype: int64


In [135]:
print(access_df['o_emp_high'].value_counts(),access_df['o_emp_med'].value_counts(),access_df['o_emp_low'].value_counts())

o_emp_high
0    513
1    265
Name: count, dtype: int64 o_emp_med
0    522
1    256
Name: count, dtype: int64 o_emp_low
0    521
1    257
Name: count, dtype: int64


In [136]:
print(10000*access_df['O_EMP_DEN'].quantile(0.66),10000*access_df['O_EMP_DEN'].quantile(0.33))

8151.803773260001 3105.9253120700005


In [21]:
# rename few columns
egress_df.rename(columns = {'E_DISTANCE_MAP':'DISTANCE', 'E_TIME_MAP':'TIME','EGRESS_SLOPE':'SLOPE','A_INTERSECTIONS':'INTERSECTIONS'
                            ,'E_SLOPE_TYPE':'SLOPE_TYPE'}, inplace = True)

In [22]:
print(egress_df[egress_df['SLOPE']>1].shape[0])
print(egress_df[egress_df['SLOPE']<-1].shape[0])
print(egress_df.shape[0]-166-121)

KeyError: 'SLOPE'

In [ ]:
first_mapping = {2: 3, 3: 3, 1: 4, 4: 4}
egress_df['RENT_BICYCLE'] = egress_df['RENT_BICYCLE'].replace(first_mapping)

# Second mapping: intermediate values to final values
second_mapping = {3: 1, 4: 2}
egress_df['RENT_BICYCLE'] = egress_df['RENT_BICYCLE'].replace(second_mapping)

# Verify the new counts
new_counts = egress_df['RENT_BICYCLE'].value_counts()

In [ ]:
egress_df['UP']=0
egress_df['DOWN']=0
egress_df['FLAT']=0

# if SLOPE is positive then make POS as 1 else 0
egress_df.loc[egress_df['SLOPE']>1,'UP']=1
egress_df.loc[egress_df['SLOPE']<-1,'DOWN']=1
egress_df.loc[(egress_df['SLOPE']>=-1) & (egress_df['SLOPE']<=1),'FLAT']=1

egress_df['VEH']=0

egress_df['POS']=0

# if slope is greater than 0 then POS is 1 else 0
egress_df.loc[egress_df['SLOPE']>0,'POS']=1

# if CARS or TWO_WHEELER is greater than 1 then mark VEH as 1 else 0
egress_df.loc[(egress_df['CARS']>1) | (egress_df['TWO_WHEELER']>1),'VEH']=1


egress_df['SHORT']=0
egress_df['MEDIUM']=0
egress_df['LONG']=0

# if DISTANCE_MAP is less than 1 then make LOW as 1
egress_df.loc[egress_df['DISTANCE']<1,'SHORT']=1
# if DISTANCE_MAP is greater than 1 and less than 3 then make MEDIUM 1
egress_df.loc[(egress_df['DISTANCE']>=1) & (egress_df['DISTANCE']<3),'MEDIUM']=1
# if DISTANCE_MAP is greater than 3 then make HIGH as 1
egress_df.loc[egress_df['DISTANCE']>=3,'LONG']=1


egress_df['AGE1']=0
egress_df['AGE2']=0
egress_df['AGE3']=0 

# if AGE_C is 1 then AGE1 is 1
egress_df.loc[egress_df['AGE_C']==1,'AGE1']=1
egress_df.loc[egress_df['AGE_C']==2,'AGE2']=1
egress_df.loc[egress_df['AGE_C']==3,'AGE3']=1

egress_df['D_POP_DEN']=egress_df['D_POP_DEN']/10000
egress_df['D_EMP_DEN']=egress_df['D_EMP_DEN']/10000
egress_df['M_POP_DEN']=egress_df['M_POP_DEN']/10000
egress_df['M_EMP_DEN']=egress_df['M_EMP_DEN']/10000


egress_df['d_pop_low']=0
egress_df['d_pop_med']=0
egress_df['d_pop_high']=0

egress_df['d_emp_high']=0
egress_df['d_emp_med']=0
egress_df['d_emp_low']=0

egress_df['m_pop_low']=0
egress_df['m_pop_med']=0
egress_df['m_pop_high']=0

egress_df['m_emp_low']=0
egress_df['m_emp_med']=0
egress_df['m_emp_high']=0

egress_df.loc[egress_df['D_POP_DEN']>egress_df['D_POP_DEN'].quantile(0.66),'d_pop_high']=1
egress_df.loc[(egress_df['D_POP_DEN']<=egress_df['D_POP_DEN'].quantile(0.66)) & (egress_df['D_POP_DEN']>egress_df['D_POP_DEN'].quantile(0.33)),'d_pop_med']=1
egress_df.loc[egress_df['D_POP_DEN']<=egress_df['D_POP_DEN'].quantile(0.33),'d_pop_low']=1

egress_df.loc[egress_df['D_EMP_DEN']>egress_df['D_EMP_DEN'].quantile(0.66),'d_emp_high']=1
egress_df.loc[(egress_df['D_EMP_DEN']<=egress_df['D_EMP_DEN'].quantile(0.66)) & (egress_df['D_EMP_DEN']>egress_df['D_EMP_DEN'].quantile(0.33)),'d_emp_med']=1
egress_df.loc[egress_df['D_EMP_DEN']<=egress_df['D_EMP_DEN'].quantile(0.33),'d_emp_low']=1

egress_df.loc[egress_df['M_POP_DEN']>egress_df['M_POP_DEN'].quantile(0.66),'m_pop_high']=1
egress_df.loc[(egress_df['M_POP_DEN']<=egress_df['M_POP_DEN'].quantile(0.66)) & (egress_df['M_POP_DEN']>egress_df['M_POP_DEN'].quantile(0.33)),'m_pop_med']=1
egress_df.loc[egress_df['M_POP_DEN']<=egress_df['M_POP_DEN'].quantile(0.33),'m_pop_low']=1

egress_df.loc[egress_df['M_EMP_DEN']>egress_df['M_EMP_DEN'].quantile(0.66),'m_emp_high']=1
egress_df.loc[(egress_df['M_EMP_DEN']<=egress_df['M_EMP_DEN'].quantile(0.66)) & (egress_df['M_EMP_DEN']>egress_df['M_EMP_DEN'].quantile(0.33)),'m_emp_med']=1
egress_df.loc[egress_df['M_EMP_DEN']<=egress_df['M_EMP_DEN'].quantile(0.33),'m_emp_low']=1





In [ ]:
egress_df['WORK']=0
egress_df.loc[(egress_df['O_TYPE']=='Work') | (egress_df['D_TYPE']=='Work'),'WORK']=1

egress_df['DEN']=0
egress_df['DEN']=egress_df['INTERSECTIONS']/egress_df['DISTANCE']

egress_df=egress_df[egress_df['DEN']<30]

egress_df['HIGH']=0
egress_df['MED']=0

# if DEN is greater than 66 percentile of data of DEN then HIGH is 1 for that entry
# if DEN is between 33 and 66 percentiles of data of DEN then MED is 1 for that entry
egress_df.loc[egress_df['DEN']>egress_df['DEN'].quantile(0.66),'HIGH']=1
egress_df.loc[(egress_df['DEN']<=egress_df['DEN'].quantile(0.66)) & (egress_df['DEN']>egress_df['DEN'].quantile(0.33)),'MED']=1

egress_df['UNO']=1

egress_df['FAV']=1
egress_df.loc[(egress_df['SLOPE_TYPE']=='rise') | (egress_df['SLOPE_TYPE']=='mixed')| (egress_df['SLOPE_TYPE']=='hydrograph'),'FAV']=0

egress_df['DIST1']=0
egress_df['DIST2']=0
egress_df['DIST3']=0
egress_df['DIST4']=0
egress_df['DIST5']=0

egress_df.loc[egress_df['DISTANCE']<1,'DIST1']=1
egress_df.loc[(egress_df['DISTANCE']>=1) & (egress_df['DISTANCE']<2),'DIST2']=1
egress_df.loc[(egress_df['DISTANCE']>=2) & (egress_df['DISTANCE']<3),'DIST3']=1
egress_df.loc[(egress_df['DISTANCE']>=3) & (egress_df['DISTANCE']<4),'DIST4']=1
egress_df.loc[(egress_df['DISTANCE']>=4),'DIST5']=1



egress_df[['ID', 'RENT_BICYCLE','MALE', 'SLOPE', 'SLOPE_TYPE', 'DISTANCE', 'AGE1','AGE2','AGE3',
           'TIME','O_TYPE','D_TYPE','EGRESS_MODE','VEH','INTERSECTIONS','WORK','UNO','D_POP_DEN','D_EMP_DEN','M_POP_DEN','M_EMP_DEN','d_pop_med','d_pop_high','d_emp_med','d_emp_high','d_pop_low','m_pop_low',
           'm_pop_low','m_pop_high','m_pop_med','m_emp_low','m_emp_med','m_emp_high','DEN',
           'UP','DOWN','FLAT','LONG','SHORT','MEDIUM','FAV','DIST1','DIST2','DIST3','DIST4','DIST5','PARK_RENT']].to_csv(path+"egress_data_analysis.csv",index=True)

In [ ]:
print(egress_df['AGE_C'].value_counts(),egress_df['MALE'].value_counts(),egress_df['WORK'].value_counts(),egress_df['PARK_RENT'].value_counts())

AGE_C
1    526
2     75
3     54
Name: count, dtype: int64 MALE
1    369
0    286
Name: count, dtype: int64 WORK
0    364
1    291
Name: count, dtype: int64 PARK_RENT
10    330
5     187
15    138
Name: count, dtype: int64


In [ ]:
print(egress_df['m_emp_high'].value_counts(),egress_df['m_emp_med'].value_counts(),egress_df['m_emp_low'].value_counts())

m_emp_high
0    447
1    208
Name: count, dtype: int64 m_emp_med
0    437
1    218
Name: count, dtype: int64 m_emp_low
0    426
1    229
Name: count, dtype: int64


In [ ]:
print(egress_df[egress_df['DISTANCE']<1].shape[0])
print(egress_df[(egress_df['DISTANCE']>=1) & (egress_df['DISTANCE']<2)].shape[0])
print(egress_df[(egress_df['DISTANCE']>=2) & (egress_df['DISTANCE']<3)].shape[0])
print(egress_df[(egress_df['DISTANCE']>=3) & (egress_df['DISTANCE']<4)].shape[0])
print(egress_df[(egress_df['DISTANCE']>=4)].shape[0])

304
169
75
26
81
